Implemented crewai to create 3 agents as mentioned in the problem


Initial Part is same as the basic_rag part


IMport crewai librarier

In [5]:
from crewai_tools import PDFSearchTool
from crewai_tools import tools
from crewai import Crew , Task , Agent


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from PyPDF2 import PdfReader
from langchain_deepseek import ChatDeepSeek

/home/sm4th/.local/lib/python3.10/site-packages/PyPDF2/__init__.py:21: DeprecationWarning: PyPDF2 is deprecated. Please move to the pypdf library instead.
  warnings.warn(


Defining the embedding and chat llm

In [7]:
llm =  OpenAIEmbeddings(
        api_key = 'api_key',
        model="text-embedding-3-small" ,
)

openai_llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    api_key='api_key',
    timeout=60
)

Creating the Agents

Information Agent

In [8]:
Information_Agent = Agent(
    role = 'Information Extractor',
    goal = 'Extract key entities, facts, and relationships from document text',
    backstory=(
        "You are an expert at extracting entities , facts and relationship between them from the given text"
        "Give your output in JSON format so that another agent can understand it"
    ),
    verbose = True,
    allow_delegation = False ,
    llm= openai_llm,
)

Synthesis Agent

In [9]:
Synthesis_Agent = Agent(
    role='summarizer',
    goal ='Summarize and organize extracted information',
    backstory = (
        "You are an expert at summarizing and organising information"
        "Use the entities , facts , relationship , etc given to you to summarize and organize them"
        "Provide a clean and concise answer"
        "Provide your Output in JSON format"
    ),
    verbose = True,
    allow_delegation = False,
    llm = openai_llm

)

Query Handling Agent

In [10]:
Query_agent = Agent(
    role = 'query handling',
    goal = 'Handle natural language questions about processed documents',
    backstory = (
        "you are an expert at answering queries regarding processed document with help of a summarised and organised text "
        "your should be proper and concise"
        "make sure answer is relavent to the question"
    ),
    verbose = True,
    allow_delegation = False,
    llm = openai_llm,
)



Defining Task for the agents

In [11]:
information_task = Task(
    description = ("Analyse the provided text : {text}"
                   "Based on the text extract entities , facts and relationships"
                   "return a clean and concise answer retaining important features of the text"
    ),
    expected_output = ("Give a JSON formatted file with Entities  , Facts and Relationships"),
    agent = Information_Agent
)


summarize_task = Task(
    description = ("Based on the response from the information task summarize and organize the information provided"
                   "make the answer clean and concise"
    ),
    expected_output = ("Give a JSON formatted file with summarized and organized text"),
    agent = Synthesis_Agent,
    context = [information_task],
)

query_task = Task(
    description = ("Based on the response from the summarize task and the question  generate a answer relavent to the question and information"
                   "make the answer proper and concise"
    ),
    expected_output = ("Answer in english for the question user asked"),
    agent = Query_agent,
    context = [summarize_task],
)

Initialising a crew of agents

In [12]:
rag_crew = Crew(
    agents = [Information_Agent, Synthesis_Agent, Query_agent],
    tasks = [information_task, summarize_task, query_task],
    verbose =True,
)

In [ ]:
inputs = {"text" : "As LLMs continue to push technological boundaries, KV Caches have proven to be indispensable in ensuring inference efficiency and scalability." }
       # just example text
results = rag_crew.kickoff(inputs=inputs)       
print(results)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 045f94b4-d928-4663-9b23-4da97db4d86c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Agent: Information Extractor

## Task: Analyse the provided text : As LLMs continue to push technological boundaries, KV Caches have proven to be
indispensable in ensuring inference efficiency and scalability.Based on the text extract entities , facts and 
relationshipsreturn a clean and concise answer retaining important features of the text

Output()